In [ ]:
# import tensorflow as tf
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

In [15]:
import pandas as pd
import numpy as np
import joblib
from sklearn.preprocessing import StandardScaler
import pickle

In [6]:
df=pd.read_csv('data/test_geometric_features_normalized.csv')
# df2=pd.read_csv('geometric_features.csv')

linearity             740
planarity             740
sphericity            740
omnivariance          740
anisotropy            740
eigenentropy          740
sum_of_eigenvalues    740
change_curvature      740
id                    740
class                  12
reflectance            19
dtype: int64
anisotropy            2438
change_curvature      2438
eigenentropy          2438
linearity             2438
omnivariance          2438
planarity             2438
sphericity            2438
sum_of_eigenvalues    2438
class                   47
reflectance             37
dtype: int64


In [23]:
#testing saved ML models 

# scaler = StandardScaler()
X_test=pd.read_csv('../X_test.csv')
y_test=pd.read_csv('../y_test.csv')
y_test=y_test['true_labels'].to_numpy()
# X_test = scaler.transform(X_test)
X_test=(X_test-X_test.mean())/X_test.std()
# print(X_test.shape)
filename = 'knn.sav'
loaded_model = joblib.load(filename)
loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, y_test)
y_pred = loaded_model.predict(X_test)
print(y_pred)

AttributeError: 'numpy.ndarray' object has no attribute 'predict'

In [ ]:
#testing saved DL models
import os
import glob

import trimesh
from plyfile import PlyData, PlyElement
import open3d as o3d

import numpy as np
import pandas as pd

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras import callbacks

from matplotlib import pyplot as plt

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, ConfusionMatrixDisplay,classification_report

import warnings
warnings.filterwarnings("ignore")

points = []
labels = []

for file in glob.glob('../data/sampled_ply/*.ply'):
# for file in glob.glob('paris_lille/sampled_ply/*.ply'):    
    
#     print(file)
    label = file.split('/')[-1].split('_')[2]
#     print(label)
    ply = PlyData.read(file)
    data = ply.elements[0].data
    data_pd = pd.DataFrame(data)
    data_np = np.zeros(data_pd.shape, dtype=np.float)
    property_names = data[0].dtype.names
    
    for i, name in enumerate(property_names):
        data_np[:,i] = data_pd[name]
        
    data_np = data_np[:,:3]    
    data_np = preprocessing.minmax_scale(data_np)
    
    points.append(data_np)
    labels.append(label)
object_classes = {
    '0' : 'Unclassified',
    '100000000' : 'Other',
    '200000000' : 'Surface',
    '201000000' : 'Other Surface',
    '202000000' : 'Ground',
    '202010000' : 'Other Ground',
    '202020000' : 'Road',
    '202030000' : 'Sidewalk',
    '202040000' : 'Curb',
    '202050000' : 'Island',
    '202060000' : 'Vegetation',
    '203000000' : 'Building',
    '300000000' : 'Object',
    '301000000' : 'Other Object',
    '302000000' : 'Static',
    '302010000' : 'Other Static',
    '302020000' : 'Punctual Object',
    '302020100' : 'Other Punctual Object',
    '302020200' : 'Post',
    '302020300' : 'Bollard',
    '302020400' : 'Floor Lamp',
    '302020500' : 'Traffic Light',
    '302020600' : 'Traffic Sign',
    '302020700' : 'Signboard',
    '302020800' : 'Mailbox',
    '302020900' : 'Trash Can',
    '302021000' : 'Meter',
    '302021100' : 'Bicycle Terminal',
    '302021200' : 'Bicycle Rack',
    '302021300' : 'Statue',
    '302030000' : 'Linear',
    '302030100' : 'Other Linear',
    '302030200' : 'Barrier',
    '302030300' : 'Roasting',
    '302030400' : 'Grid',
    '302030500' : 'Chain',
    '302030600' : 'Wire',
    '302030700' : 'Low Wall',
    '302040000' : 'Extended',
    '302040100' : 'Other Extended',
    '302040200' : 'Shelter',
    '302040300' : 'Kiosk',
    '302040400' : 'Scaffold',
    '302040500' : 'Bench',
    '302040600' : 'Distribution Box',
    '302040700' : 'Lighting Console',
    '302040800' : 'Windmill',
    '303000000' : 'Dynamic',
    '303010000' : 'Other Dynamic',
    '303020000' : 'Pedestrian',
    '303020100' : 'Other Pedestrian',
    '303020200' : 'Still Pedestrian',
    '303020300' : 'Walking Pedestrian',
    '303020400' : 'Running Pedestrian',
    '303020500' : 'Stroller Pedestrian',
    '303020600' : 'Holding Pedesterian',
    '303020700' : 'Leaning Pedestrian',
    '303020800' : 'Skater',
    '303020900' : 'Rollerskater',
    '303021000' : 'Wheelchair',
    '303030000' : '2 Wheelers',
    '303030100' : 'Other 2 Wheels',
    '303030200' : 'Bicycle',
    '303030201' : 'Other Bicycle',
    '303030202' : 'Mobile Bicycle',
    '303030203' : 'Stopped Bicycle',
    '303030204' : 'Parked Bicycle',
    '303030300' : 'Scooter',
    '303030301' : 'Other Scooter',
    '303030302' : 'Mobile Scooter',
    '303030303' : 'Stopped Scooter',
    '303030304' : 'Parked Scooter',
    '303030400' : 'Moped',
    '303030401' : 'Other Moped',
    '303030402' : 'Mobile Moped',
    '303030403' : 'Stopped Moped',
    '303030404' : 'Parked Moped',
    '303030500' : 'Motorbike',
    '303030501' : 'Other Motorbike',
    '303030502' : 'Mobile Motorbike',
    '303030503' : 'Stopped Motorbike',
    '303030504' : 'Parked Motorbike',
    '303040000' : '4+ Wheelers',
    '303040100' : 'Other 4+ Wheelers',
    '303040200' : 'Car',
    '303040201' : 'Other Car',
    '303040202' : 'Mobile Car',
    '303040203' : 'Stopped Car',
    '303040204' : 'Parked Car',
    '303040300' : 'Van',
    '303040301' : 'Other Van',
    '303040302' : 'Mobile Van',
    '303040303' : 'Stopped Van',
    '303040304' : 'Parked Van',
    '303040400' : 'Truck',
    '303040401' : 'Other Truck',
    '303040402' : 'Mobile Truck',
    '303040403' : 'Stopped Truck',
    '303040404' : 'Parked Truck',
    '303040500' : 'Bus',
    '303040501' : 'Other Bus',
    '303040502' : 'Mobile Bus',
    '303040503' : 'Stopped Bus',
    '303040504' : 'Parked Bus',
    '303050000' : 'Furniture',
    '303050100' : 'Other Furniture',
    '303050200' : 'Table',
    '303050300' : 'Chair',
    '303050400' : 'Stool',
    '303050500' : 'Trash Can',
    '303050600' : 'Waste',
    '304000000' : 'Natural',
    '304010000' : 'Other Natural',
    '304020000' : 'Tree',
    '304030000' : 'Bush',
    '304040000' : 'Potted Plant',
    '304050000' : 'Hedge'
}


# In[64]:


labels = [object_classes.get(item,item) for item in labels]
unique_labels = np.unique(labels)

mapping = {}
keys = range(len(unique_labels))

for i in keys:
    mapping[i] = unique_labels[i]
    
rev_mapping = dict((v,k) for k,v in mapping.items())
new_labels = [rev_mapping[i] for i in labels]


# In[102]:


## testing KP
# print(keys)
# print(unique_labels[0])
# print(rev_mapping)
# print(new_labels)
# print(labels)


# In[66]:


NUM_POINTS = 1000
NUM_CLASSES = 48
BATCH_SIZE = 32


# In[67]:


# checking if gpu is available or not KP
# print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print(tf.__version__)


# In[68]:


X_train, X_test, y_train, y_test = train_test_split(points, new_labels, test_size=.33, random_state=0)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=.15, random_state=0)

train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(BATCH_SIZE)
val_dataset = tf.data.Dataset.from_tensor_slices((X_val, y_val)).batch(BATCH_SIZE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(BATCH_SIZE)


model.load_weights(checkpoint_path)

# Re-evaluate the model
loss, acc = model.evaluate(test_images, test_labels, verbose=2)
print("Restored model, accuracy: {:5.2f}%".format(100 * acc))